In [55]:
import requests
import uuid
import base64
import random
import json

In [3]:
LINKEDIN_BASE_URL = "https://www.linkedin.com"
API_BASE_URL = f"{LINKEDIN_BASE_URL}/voyager/api"

REQUEST_HEADERS = {
        "user-agent": " ".join(
            [
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5)",
                "AppleWebKit/537.36 (KHTML, like Gecko)",
                "Chrome/83.0.4103.116 Safari/537.36",
            ]
        ),
        # "accept": "application/vnd.linkedin.normalized+json+2.1",
        "accept-language": "en-AU,en-GB;q=0.9,en-US;q=0.8,en;q=0.7",
        "x-li-lang": "en_US",
        "x-restli-protocol-version": "2.0.0",
        # "x-li-track": '{"clientVersion":"1.2.6216","osName":"web","timezoneOffset":10,"deviceFormFactor":"DESKTOP","mpName":"voyager-web"}',
    }

AUTH_REQUEST_HEADERS = {
        "X-Li-User-Agent": "LIAuthLibrary:3.2.4 \
                            com.linkedin.LinkedIn:8.8.1 \
                            iPhone:8.3",
        "User-Agent": "LinkedIn/8.8.1 CFNetwork/711.3.18 Darwin/14.0.0",
        "X-User-Language": "en",
        "X-User-Locale": "en_US",
        "Accept-Language": "en-us",
    }
cookies = {
'bcookie': ,
 'lang': ,
 'liap': 'true',
 'lidc': ,
 'JSESSIONID': ,
 'bscookie': ,
 'li_at': 
}
proxies = {}

In [65]:
session = requests.session()
session.proxies.update(proxies)
session.headers.update(REQUEST_HEADERS)

session.cookies.update(cookies)
session.headers["csrf-token"] = session.cookies["JSESSIONID"].strip('"')

In [72]:
def _fetch(uri, **kwargs):
    url = f"{API_BASE_URL}{uri}"
    return session.get(url, **kwargs)

def _post(uri, **kwargs):
    url = f"{API_BASE_URL}{uri}"
    return session.post(url, **kwargs)

def generate_trackingId_as_charString():
    """Generates and returns a random trackingId
    :return: Random trackingId string
    :rtype: str
    """
    random_int_array = [random.randrange(256) for _ in range(16)]
    rand_byte_array = bytearray(random_int_array)
    
    return "".join([chr(i) for i in rand_byte_array])

In [73]:
my_message = "sup"
message_event = {
"eventCreate": {
    "originToken": str(uuid.uuid4()),
    "value": {
        "com.linkedin.voyager.messaging.create.MessageCreate": {
            "attributedBody": {
                "text": my_message,
                "attributes": [],
            },
            "attachments": [],
        }
    },
    "trackingId": generate_trackingId_as_charString(),
},
"dedupeByClientGeneratedToken": False,
}
send_message = _post(
                f"/messaging/conversations/{conversation_urn}/events",
                params={"action": "create"},
                data=json.dumps(message_event))
send_message

<Response [201]>

In [14]:
def get_profile_data(data):
    profile = data["profile"]
    if "miniProfile" in profile:
        if "picture" in profile["miniProfile"]:
            profile["displayPictureUrl"] = profile["miniProfile"]["picture"][
                "com.linkedin.common.VectorImage"
            ]["rootUrl"]

            images_data = profile["miniProfile"]["picture"][
                "com.linkedin.common.VectorImage"
            ]["artifacts"]
            '''
            for img in images_data:
                w, h, url_segment = itemgetter(
                    "width", "height", "fileIdentifyingUrlPathSegment"
                )(img)
                profile[f"img_{w}_{h}"] = url_segment
            '''

        profile["profile_id"] = profile["miniProfile"]["entityUrn"]
        profile["profile_urn"] = profile["miniProfile"]["entityUrn"]
        profile["member_urn"] = profile["miniProfile"]["objectUrn"]
        profile["public_id"] = profile["miniProfile"]["publicIdentifier"]

        del profile["miniProfile"]

    del profile["defaultLocale"]
    del profile["supportedLocales"]
    del profile["versionTag"]
    del profile["showEducationOnProfileTopCard"]

    # massage [experience] data
    experience = data["positionView"]["elements"]
    for item in experience:
        if "company" in item and "miniCompany" in item["company"]:
            if "logo" in item["company"]["miniCompany"]:
                logo = item["company"]["miniCompany"]["logo"].get(
                    "com.linkedin.common.VectorImage"
                )
                if logo:
                    item["companyLogoUrl"] = logo["rootUrl"]
            del item["company"]["miniCompany"]

    profile["experience"] = experience

    # massage [education] data
    education = data["educationView"]["elements"]
    for item in education:
        if "school" in item:
            if "logo" in item["school"]:
                item["school"]["logoUrl"] = item["school"]["logo"][
                    "com.linkedin.common.VectorImage"
                ]["rootUrl"]
                del item["school"]["logo"]

    profile["education"] = education

    # massage [languages] data
    languages = data["languageView"]["elements"]
    for item in languages:
        del item["entityUrn"]
    profile["languages"] = languages

    # massage [publications] data
    publications = data["publicationView"]["elements"]
    for item in publications:
        del item["entityUrn"]
        for author in item.get("authors", []):
            del author["entityUrn"]
    profile["publications"] = publications

    # massage [certifications] data
    certifications = data["certificationView"]["elements"]
    for item in certifications:
        del item["entityUrn"]
    profile["certifications"] = certifications

    # massage [volunteer] data
    volunteer = data["volunteerExperienceView"]["elements"]
    for item in volunteer:
        del item["entityUrn"]
    profile["volunteer"] = volunteer

    # massage [honors] data
    honors = data["honorView"]["elements"]
    for item in honors:
        del item["entityUrn"]
    profile["honors"] = honors

    # massage [projects] data
    projects = data["projectView"]["elements"]
    for item in projects:
        del item["entityUrn"]
    profile["projects"] = projects
    
    return profile